# duie 数据预处理

In [3]:
import numpy as np
import os
import torch

from my_py_toolkit.file.file_toolkit import readjson, writejson
from my_py_toolkit.data_visulization.matplotlib import plot_coh

In [5]:
# 参数
data_dir = '../resources/dataset/duie'
data_files = ['train_data.json', 'dev_data.json']
schemas_file = 'all_50_chemas'


In [6]:
# 读取所有数据
def readdata(paths):
    datas = []
    for p in paths:
        datas.extend(readjson(p))
    return datas

datas = readdata([os.path.join(data_dir, f) for f in data_files])
len(datas)



JSONDecodeError: Extra data: line 2 column 1 (char 828)